In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=bdda01cebf1b846d9fdd72f4f99ccaad9914164851eaa0306b6d4e36e1945414
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.streaming import *

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("StreamingExample") \
    .getOrCreate()

In [7]:
print(spark)

In [8]:
!pip install kaggle


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
!mkdir ~/.kaggle/
!touch ~/.kaggle/kaggle.json

In [10]:
api_token = {"username":"dickyindrakuncahyo","key":"c7d0c46663f257110080794b3a2914bc"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json


In [14]:
!kaggle datasets download -d dickyindrakuncahyo/input-5025201250

  0% 0.00/5.42k [00:00<?, ?B/s]
100% 5.42k/5.42k [00:00<00:00, 4.23MB/s]


In [20]:
!unzip input-5025201250.zip -d /input-5025201250

Archive:  input-5025201250.zip
checkdir:  cannot create extraction directory: path/to/input-5025201250
           No such file or directory


In [21]:
!ls -la

total 44
drwxr-xr-x 1 root root 4096 Apr 14 01:53 .
drwxr-xr-x 1 root root 4096 Apr 14 01:55 ..
drwxr-xr-x 4 root root 4096 Apr 12 13:33 .config
-rw-r--r-- 1 root root 5545 Apr 14 01:52 input-5025201250.zip
-rw-r--r-- 1 root root 2239 Apr 14 01:48 news-1.json
-rw-r--r-- 1 root root 4320 Apr 14 01:48 news-2.json
-rw-r--r-- 1 root root 5076 Apr 14 01:48 news-3.json
drwxr-xr-x 1 root root 4096 Apr 12 13:34 sample_data


In [23]:
input_path = "/input-5025201250"
output_path = "/output-5025201250"

df = spark \
  .readStream \
  .schema(StructType([
      StructField("title", StringType()),
      StructField("content", StringType()),
      StructField("category", StringType()),
      StructField("publisher", StringType())
  ])) \
  .option("maxFilesPerTrigger", 1) \
  .json(input_path)

In [33]:
df_processed = df \
  .withColumn("word_count", size(split(col("content"), " "))) \
  .withColumn("date", current_date()) \
  .select("title", "category", "publisher", "word_count", "date")

In [37]:
query = df_processed \
  .writeStream \
  .outputMode("append") \
  .option("checkpointLocation", "/tmp/checkpoint") \
  .option("path", output_path) \
  .start()



In [38]:
df_processed.printSchema()

root
 |-- title: string (nullable = true)
 |-- category: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- word_count: integer (nullable = false)
 |-- date: date (nullable = false)



In [ ]:
query.awaitTermination()